In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.activations import linear, relu, sigmoid
from modules.utils import filter_data_by_year_month, create_features_from_past, create_X_y



In [2]:
X = pd.read_csv('/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT')


In [3]:
YEARS = [2019,2021,2022,2023,2024]

Data = filter_data_by_year_month(X, YEARS)

Data1 = create_features_from_past(Data,['close', 'open', 'high', 'low', 'volume'], 4)

In [ ]:
Data1.head()

In [ ]:
num_nan_rows = Data1.isna().any(axis=1).sum()
print(num_nan_rows)

In [4]:
X,y = create_X_y(Data1)

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
y.head()

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [6]:
# Calculate sizes for train and validation sets
dataset_size = len(X)  # Total number of samples in your dataset
train_size = int(0.8 * dataset_size)

# Shuffle the dataset if needed
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset
train_dataset = dataset.take(train_size)       # Take the first 80%
val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# Batch both datasets if needed
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [7]:
tf.random.set_seed(1234) # for consistent results
model = Sequential([
    Dense(64, activation='relu', input_shape=(20,)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')
])


/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

In [ ]:
model = tf.keras.models.load_model("../models/RNN_model_1.h5") #IF you need to dowload a model 

In [8]:
model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['mae']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_mae',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=6,          # Number of epochs with no improvement before reducing
    min_lr=1e-6          # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/RNN_model_1.keras',             # File to save the best model
    monitor='val_mae',       # Metric to monitor for improvement
    mode='min',                   # Mode 'min' for mae (since lower is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)

history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=50,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - loss: 104361536.0000 - mae: 6784.2158
Epoch 1: val_mae improved from inf to 8555.78711, saving model to ../models/RNN_model_1.keras
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 5s 978us/step - loss: 104358208.0000 - mae: 6784.1230 - val_loss: 100846608.0000 - val_mae: 8555.7871 - learning_rate: 0.0100
Epoch 2/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - loss: 80861640.0000 - mae: 6043.8276
Epoch 2: val_mae did not improve from 8555.78711
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 4s 933us/step - loss: 80861288.0000 - mae: 6043.8154 - val_loss: 197295152.0000 - val_mae: 11923.4023 - learning_rate: 0.0100
Epoch 3/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: 81575424.0000 - mae: 6059.4692
Epoch 3: val_mae did not improve from 8555.78711
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 4s 937us/step - loss: 81575112.0000 - mae: 6059.4590 - val_loss: 292525248.0000 - val_mae: 14524.1436 - learning_rate: 0.0100
Epoch 4/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/

In [9]:
model.save("../models/RNN_model_1.h5")  # Saves the model as an HDF5 file
